# Urban Traffic Palm Springs Convention Center
Simulate the urban traffic from the city of Palm Springs nearby the Convention Center where the Esri Developer Summit takes place using [ArcGIS API for Python](https://developers.arcgis.com/python/).

In [ ]:
# author: Jan Tschada
# SPDX-License-Identifer: Apache-2.0

## Required Python Modules
You need to install ArcGIS API for Python, please follow the [guide](https://developers.arcgis.com/python/guide/anaconda/).

In [ ]:
from arcgis.gis import GIS
from arcgis.features import FeatureSet
from datetime import date, datetime, timedelta
from georapid.client import GeoRapidClient
from georapid.factory import EnvironmentClientFactory
from geourban.services import aggregate, query, simulations, top
from geourban.types import GridType, VehicleType

## Connect to ArcGIS Online

In [ ]:
gis = GIS()

## Create a map showing the city of Palm Springs

In [ ]:
palmsprings_map = gis.map('Palm Springs, United Staates', zoomlevel=13)
palmsprings_map

## Connect to geourban services
The host parameter must target the specific host like "geoprotests.p.rapidapi.com". Furthermore, the factory directly access `os.environ['x_rapidapi_key']` and uses the specified API key as a header parameter. Otherwise, `georapid.factory.EnvironmentClientFactory.create_client_with_host()` will raise a `ValueError`.

Please, check out the [RapidAPI Account Creation and Management Guide](https://docs.rapidapi.com/docs/account-creation-and-settings).

In [ ]:
client = EnvironmentClientFactory.create_client_with_host('geourban.p.rapidapi.com')

## List the available simulations

In [ ]:
urban_simulations = simulations(client)
urban_simulations

## Request the top 5 accumulated car traffic grid cells
We request these hotspots for the city of Palm Springs by using the urban region code `USQ115930`, the simulation date `2024-03-07`, the vehicle type `Car`, and the grid type `agent`. The returned GeoJSON features represents the grid cells with the highest car throughput.

In [ ]:
palmsprings_region_code = 'USQ115930'
simulation_date = date(2024, 3, 7)
vehicle_type = VehicleType.CAR
grid_type = GridType.AGENT
limit = 5
top_traffic_grid_cells = top(client, palmsprings_region_code, simulation_date, vehicle_type, grid_type, limit=limit)
top_traffic_grid_cells

## Convert the returned GeoJSON result into a FeatureSet
The FeatureSet offers direct access to a spatially enabled dataframe. We can easily inspect the time frames (`start_time` - `end_time`) and the number of car vehicles `agent_count`.

We see that the highest car throughput is before and after the conference event, e.g. between `07:00:00 - 07:59:59` and `18:00:00 - 18:59:59`.

In [ ]:
top_traffic_fset = FeatureSet.from_geojson(top_traffic_grid_cells)
top_traffic_sdf = top_traffic_fset.sdf
top_traffic_sdf

## Map the traffic grid cells

In [ ]:
top_traffic_sdf.spatial.plot(palmsprings_map, renderer_type='s', colors='#E80000', alpha=0.3)
palmsprings_map

## Query the simulated agents nearby
We are using the center of the hotspots near the Palm Springs International Airport, request the simulated agents being within a distance of `250 meters`, and specify a `30 seconds` time window starting from `07:30:00`.

In [ ]:
simulation_datetime = datetime.fromisoformat('2024-03-07T07:30:00')
(latitude, longitude) = (33.822246, -116.494852)
(seconds, meters) = (30, 250)
car_positions = query(client, simulation_datetime, vehicle_type, latitude, longitude, seconds, meters)
car_positions_fset = FeatureSet.from_geojson(car_positions)
car_positions_sdf = car_positions_fset.sdf
car_positions_sdf

## Map the car positions nearby

In [ ]:
palmsprings_map = gis.map('Palm Springs, United Staates', zoomlevel=13)
palmsprings_map.zoom_to_layer(car_positions_sdf)
car_positions_sdf.spatial.plot(palmsprings_map, renderer_type='s', colors='#E80000', marker_size=7, alpha=0.3)
palmsprings_map

## Do some data engineering
We are interested in the movements of a specific agent and want to narrow down its movement behavior.
The GeoJSON properties are treated as strings. So that we should convert the `trip` and `person` columns to `int32`. Furthermore, we convert the `trip_time` to `datetime`.

In [ ]:
import pandas as pd

In [ ]:
car_positions_sdf = car_positions_sdf.astype({'trip':'int32', 'person':'int32'}) 
car_positions_sdf[['trip_time']] = car_positions_sdf[['trip_time']].apply(pd.to_datetime)
car_positions_sdf

In [ ]:
person_1161_positions_sdf = car_positions_sdf[car_positions_sdf['person']==1161]
person_1161_positions_sdf

## Map the car positions of one specific agent

In [ ]:
palmsprings_map = gis.map('Palm Springs, United Staates', zoomlevel=13)
palmsprings_map.zoom_to_layer(person_1161_positions_sdf)
person_1161_positions_sdf.spatial.plot(palmsprings_map, renderer_type='s', colors='#E80000', marker_size=7, alpha=0.3)
palmsprings_map

## Accumulate the speed of car traffic between 08:00 and 09:00 AM

In [ ]:
palmsprings_region_code = 'USQ115930'
simulation_datetime = datetime.fromisoformat('2024-03-07T08:00:00')
vehicle_type = VehicleType.CAR
grid_type = GridType.SPEED
car_speed_cells = aggregate(client, palmsprings_region_code, simulation_datetime, vehicle_type, grid_type)
car_speed_fset = FeatureSet.from_geojson(car_speed_cells)
car_speed_sdf = car_speed_fset.sdf
car_speed_sdf

## Map the accumulated speed of car traffic

In [ ]:
palmsprings_map = gis.map('Palm Springs, United Staates', zoomlevel=13)
car_speed_sdf.spatial.plot(palmsprings_map, renderer_type='c', method='esriClassifyNaturalBreaks', class_count=5, col='speed_mean', cmap='RdYlGn')
palmsprings_map

## Map the top 5 accumulated pedestrian traffic grid cells
We request these hotspots for the city of Palm Springs by using the urban region code `USQ115930`, the simulation date `2024-03-07`, the vehicle type `Pedestrian`, and the grid type `agent`. The returned GeoJSON features represents the grid cells with the highest pedestrian throughput.

In [ ]:
palmsprings_region_code = 'USQ115930'
simulation_date = date(2024, 3, 7)
vehicle_type = VehicleType.PEDESTRIAN
grid_type = GridType.AGENT
limit = 5
top_traffic_grid_cells = top(client, palmsprings_region_code, simulation_date, vehicle_type, grid_type, limit=limit)
top_traffic_fset = FeatureSet.from_geojson(top_traffic_grid_cells)
top_traffic_sdf = top_traffic_fset.sdf
palmsprings_map = gis.map('Palm Springs, United Staates', zoomlevel=13)
top_traffic_sdf.spatial.plot(palmsprings_map, renderer_type='s', colors='#E80000', alpha=0.3)
palmsprings_map

## Map the car traffic nearby the convention center at 06:40 PM
At this time people leaving the convention center by walking to parking lots and driving away by car.

In [ ]:
simulation_datetime = datetime.fromisoformat('2024-03-07T18:40:00')
vehicle_type = VehicleType.CAR
(latitude, longitude) = (33.825868, -116.538317)
(seconds, meters) = (1, 1000)
car_positions = query(client, simulation_datetime, vehicle_type, latitude, longitude, seconds, meters)
car_positions_fset = FeatureSet.from_geojson(car_positions)
car_positions_sdf = car_positions_fset.sdf
car_positions_sdf.spatial.plot(palmsprings_map, renderer_type='s', colors='#E80000', marker_size=7, alpha=0.3)
palmsprings_map